In [1]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from scipy.sparse.csgraph import shortest_path
from scipy.linalg import expm
from scipy.spatial.distance import cdist
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from tqdm import tqdm

In [2]:
# Read the tract length of tractography (using for constructing the consensus matrix)

t_subjs = set(pd.read_csv(r"D:\Download\SFC\SFC_HCP\4_Validation\individual_VGAE\sfc_network_df.csv").Subject.apply(lambda x: x[:6]))
h5_path = r"D:\Download\SFC\SFC_HCP\2_Preprocess\1_SC\sc_matlab\individualConnectivity_tractLength.mat"
lengths = []
with h5py.File(h5_path, "r") as f:
    subjects = [str(subject) for subject in f['subjectIDs'][0]]
    for i, subject in enumerate(subjects):
        if subject in t_subjs:
            lengths.append(f['rawTractLengths'][i].T)
lengths = np.array(lengths)

In [3]:
# This function is used to construct a consensus matrix (reference: https://doi.org/10.1073/pnas.1903403116)
# But because we need individual sfc by regression approach, it was not used.

def build_consensus_matrix(sc, lengths):

    n_subjects = sc.shape[0]

    consensus_sc = np.zeros((360, 360))
    
    densities = []
    for s in range(n_subjects):
        upper_tri = sc[s][np.triu_indices(360, k=1)]
        densities.append(np.mean(upper_tri))
    n_bins = int(np.sqrt(np.mean(densities) * n_subjects))
    n_bins = max(2, n_bins)
    
    for connection_type in ['intra_left', 'intra_right', 'inter']:
        if connection_type == 'intra_left':
            mask_i, mask_j = np.meshgrid(range(180), range(180), indexing='ij')
            mask = mask_i < mask_j
        elif connection_type == 'intra_right':
            mask_i, mask_j = np.meshgrid(range(180, 360), range(180, 360), indexing='ij')
            mask = mask_i < mask_j
        else:
            mask_i, mask_j = np.meshgrid(range(180), range(180, 360), indexing='ij')
            mask = np.ones_like(mask_i, dtype=bool)
    
        connections = []
        for i, j in zip(mask_i[mask], mask_j[mask]):
            node_i, node_j = i, j
            frequency = np.sum([sc[s, node_i, node_j] for s in range(n_subjects)])
            if frequency > 0:
                avg_length = np.mean([lengths[s, node_i, node_j] for s in range(n_subjects) if sc[s, node_i, node_j] > 0])
                connections.append({
                    'i': node_i,
                    'j': node_j,
                    'frequency': frequency,
                    'length': avg_length
                })
        if not connections:
            continue
    
        connection_lengths = [conn['length'] for conn in connections]
        bin_edges = stats.mstats.mquantiles(connection_lengths, np.linspace(0, 1, n_bins + 1))
        bin_edges[0] = min(connection_lengths) - 1e-6
        bin_edges[-1] = max(connection_lengths) + 1e-6
        
        binned_connections = [[] for _ in range(n_bins)]
        for conn in connections:
            bin_idx = np.digitize(conn['length'], bin_edges) - 1
            bin_idx = max(0, min(bin_idx, n_bins - 1))
            binned_connections[bin_idx].append(conn)
            
        for bin_idx, bin_conns in enumerate(binned_connections):
            if not bin_conns:
                continue
                
            bin_frequencies = [conn['frequency'] for conn in bin_conns]
            k = int(np.mean(bin_frequencies))
            k = max(1, k)
            
            bin_conns_sorted = sorted(bin_conns, key=lambda x: x['frequency'], reverse=True)
            selected_conns = bin_conns_sorted[:k]
            for conn in selected_conns:
                consensus_sc[conn['i'], conn['j']] = 1
                consensus_sc[conn['j'], conn['i']] = 1

    return consensus_sc

In [4]:
# 1 Read sc

scs = []
for root, dirs, files in os.walk(r"D:\Download\SFC\SFC_HCP\3_Modeling\2_modeling\sc_dataset\raw"):
    for file in files:
        sc = np.load(os.path.join(root, file))
        
        mask = np.eye(sc.shape[0], dtype=bool)
        sc[mask] = 0
        threshold = np.quantile(sc[~mask], 0.932)
        sc[sc < threshold] = 0
        sc[sc >= threshold] = 1
        
        scs.append(sc)
scs = np.array(scs)

In [5]:
# 2 Read fc

fcs = []
for root, dirs, files in os.walk(r"D:\Download\SFC\SFC_HCP\3_Modeling\2_modeling\fc_dataset\raw"):
    for file in files:
        path_fc = os.path.join(root, file)
        fcs.append(np.load(path_fc))
fcs = np.array(fcs)

In [6]:
# 3 Obtain the centroid coordinates for calculating the distance between brain regions

centroids = np.load(r"D:\Download\SFC\SFC_HCP\3_Modeling\1_get_centroids\centroids\HCP_MMP1_centroids.npy")
euclidean_distance = cdist(centroids, centroids, metric='euclidean')
mask = np.eye(360, dtype=bool)
euclidean_distance = (euclidean_distance - euclidean_distance[~mask].mean()) / euclidean_distance[~mask].std()

In [7]:
# This function is used to obtain the shortest path and communicability (used as independent variable)

def getX(sc):
    path_length = shortest_path(sc, directed=False, method='D', unweighted=True)
    path_length[path_length == np.inf] = 361
    path_length = (path_length - path_length[~mask].mean()) / path_length[~mask].std()
    max_degree = np.max(np.sum(sc, axis=1))
    communicability = np.log(expm(sc / max_degree) + 1e-8)
    communicability = (communicability - communicability[~mask].mean()) / communicability[~mask].std()
    return path_length, communicability

In [8]:
# Calculate group sfc (but not used for contrast)

consensus_fc = fcs.mean(axis=0)

consensus_sc = build_consensus_matrix(scs, lengths)

consensus_path_length, consensus_communicability = getX(consensus_sc)

consensus_regression_sfc = np.zeros(360)

for node_i in range(360):
    
    y_train = np.delete(consensus_fc[node_i, :], node_i)
    X_euclidean_train = np.delete(euclidean_distance[node_i, :], node_i)
    X_path_train = np.delete(consensus_path_length[node_i, :], node_i)
    X_comm_train = np.delete(consensus_communicability[node_i, :], node_i)
    
    X_train = np.column_stack([X_euclidean_train, X_path_train, X_comm_train])
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    
    r2 = r2_score(y_train, y_pred)
    n, p = 359, 3
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

    consensus_regression_sfc[node_i] = adjusted_r2

In [9]:
# 4 Calculate individual sfc

regression_sfc = np.zeros((3992, 360))

for s in tqdm(range(998)):
    path_length, communicability = getX(scs[s])
    
    for i in range(4):
        fc = fcs[4*s+i]
        
        for node_i in range(360):
            
            y_train = np.delete(fc[node_i, :], node_i)
            X_euclidean_train = np.delete(euclidean_distance[node_i, :], node_i)
            X_path_train = np.delete(path_length[node_i, :], node_i)
            X_comm_train = np.delete(communicability[node_i, :], node_i)
            
            X_train = np.column_stack([X_euclidean_train, X_path_train, X_comm_train])
            
            model = LinearRegression()
            model.fit(X_train, y_train)
            y_pred = model.predict(X_train)
            
            r2 = r2_score(y_train, y_pred)
            n, p = 359, 3
            adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
            
            regression_sfc[4*s+i, node_i] = adjusted_r2

100%|██████████████████████████████████████████████████████████████████████████████| 998/998 [1:41:17<00:00,  6.09s/it]


In [14]:
vgcl_sfc = pd.read_csv(r"D:\Download\SFC\SFC_HCP\6_Contrast\vgcl_individual_sfc.csv")

In [19]:
regr_sfc = vgcl_sfc.copy()
regr_sfc.iloc[:, 3:] = regression_sfc

In [22]:
regr_sfc.to_csv('D:\\Download\\regr_individual_sfc.csv', index=False)